In [1]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import operator
from tqdm import tqdm
import os
import json

In [2]:
with open ("big_musician_overall_colloc_similairities.json", 'r', encoding = 'utf-8') as f:
    colloc_sim_db = json.load(f)

In [3]:
colloc_sim_db['международный благотворительный']

[[{'freqence_similarity': 0.908053607225092,
   'marked_colloc_db_index': 44,
   'sim_colloc': 'санктпетербургский государственный академический',
   'w2v_sim': 0.737537937659166},
  0.7716410715723513]]

In [4]:
from text_processing import get_text_map


C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
USER_NAME = "enze_big_musician"

In [13]:
texts_lenta = pd.read_csv("./articles/music_lenta.csv")
texts_lenta.iloc[16]['text']

"Австралийские электронные музыканты The Avalanches выпустили видео на песню Because I'm Me. Клип, снятый режиссером Грегом Бранкаллой, рассказывает о мальчике, который влюблен в работницу метрополитена и пытается привлечь ее внимание при помощи песни. Because I'm Me — четвертый сингл с альбома The Avalanches 2016 года Wildflower, который стал их первой пластинкой за 16 лет."

In [14]:
text_db = pd.read_csv(USER_NAME + "_text_db.csv",header = None)
text_db.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.6118,0.75,0.10,0.166000,0.2,0.200000,0.024267,0.030002,0.000000,0.071312,0.191511,1.0
1,0.6118,0.75,0.10,0.166000,0.2,0.200000,0.024267,0.030002,0.000000,0.071312,0.191511,1.0
2,0.6118,0.75,0.10,0.166000,0.2,0.200000,0.024267,0.030002,0.000000,0.071312,0.191511,1.0
3,0.8013,0.63,0.14,0.180714,0.0,0.071429,0.015801,0.032847,0.004245,0.145502,0.212638,1.0
4,0.8013,0.63,0.14,0.180714,0.0,0.071429,0.015801,0.032847,0.004245,0.145502,0.212638,0.5


In [15]:
sent_db = pd.read_csv(USER_NAME + "_sentence_db.csv",header = None)
sent_db.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.209091,1.0
1,0.018519,0.000000,0.021353,0.000000,0.0,0.042707,0.347170,1.0
2,0.034483,0.034483,0.000000,0.063484,0.0,0.143760,0.217857,1.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.177674,0.219048,1.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.107480,0.205263,0.0


In [16]:
word_db = pd.read_csv(USER_NAME +  "_word_db.csv",header = None)
word_db.head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,632.0,155.1,0.019617,0.027856,-0.098140,0.018417,-0.060545,0.052610,-0.008416,0.019377,...,-0.045566,-0.019588,-0.051453,-0.000417,0.015935,0.018377,-0.049183,0.040319,0.087135,1.0
1,11.0,17.1,-0.059523,0.012793,0.034546,-0.019653,-0.059405,-0.108276,-0.020982,0.038993,...,0.080227,0.072730,0.008212,0.018211,-0.058539,-0.046240,-0.067024,-0.060923,-0.018587,1.0
2,186.0,483.3,-0.012936,-0.003571,-0.036513,-0.125645,-0.025398,-0.118884,-0.007071,-0.008625,...,0.069345,0.021790,-0.002086,0.007569,-0.078134,0.048388,-0.009063,0.001203,-0.019108,1.0
3,55570.0,31374.2,0.050138,-0.029846,-0.002434,-0.010027,-0.056048,-0.110441,0.001683,-0.021618,...,0.068040,0.000184,0.093264,-0.029066,-0.021875,0.020433,-0.024090,0.030871,-0.060288,1.0
4,1275.0,725.7,0.055334,0.054794,-0.044653,0.106381,-0.044946,0.002045,0.086079,-0.024620,...,-0.036311,0.047815,0.094624,0.068983,-0.046637,-0.024489,-0.087214,-0.100475,0.059830,1.0


with open ("text_map_improved_example.json", "r", encoding = "utf-8") as f:
    examined_text = json.load(f)

In [56]:
words_dict = {}
with open(USER_NAME + "_colloc_lit.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        sp_line = line.split("_")
        #print(sp_line)
        ngramm = sp_line[0]
        freq  = sp_line[-2]
        words_dict[ngramm] = int(freq)
        #words_dict.append(line[:-1])

In [57]:
words_dict["физический сила"]

6

In [58]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted(x.items(), key=lambda kv: kv[1])

[(0, 0), (2, 1), (1, 2), (4, 3), (3, 4)]

In [59]:
sorted_x = sorted(words_dict.items(),key=lambda kv: kv[1])
sorted_x


[('задрапировать', 1),
 ('недостра', 1),
 ('илларионов', 1),
 ('ручить', 1),
 ('керамический', 1),
 ('школу', 1),
 ('энджи', 1),
 ('конный', 1),
 ('по крайний мера', 1),
 ('презирать', 1),
 ('зевавать', 1),
 ('разлаживаться', 1),
 ('мой жизнь', 1),
 ('hearsay', 1),
 ('carried', 1),
 ('утопия', 1),
 ('помокать', 1),
 ('городской комитет', 2),
 ('коллективный письмо', 2),
 ('санктпетербургский государственный академический', 2),
 ('петербургский комитет', 2),
 ('музыкальный совет', 2),
 ('музыкант журналист', 2),
 ('бывший жена певец', 2),
 ('театр новый опера', 2),
 ('многие язык', 2),
 ('английский немецкий', 2),
 ('недочет', 2),
 ('благодарный слушатель', 2),
 ('гитара нужный', 2),
 ('менеджер канадский певица', 2),
 ('раком', 2),
 ('слово агент организатор', 2),
 ('регламент мероприятие', 2),
 ('грушинки', 2),
 ('денежный средства', 2),
 ('отсутствие представитель консульство', 2),
 ('оскорбительно', 2),
 ('концерт мастер', 2),
 ('открытый урок', 2),
 ('колхозница', 2),
 ('коллектив 

In [ ]:
my_list = [ ('John', 40), ('Dana', 35), ('Betty', 10), ('Robby', 8), ('John', 20) ]
my_list.sort(key = operator.itemgetter(1))
print(my_list)

In [19]:
with open ("lyashevskaya_freq_dict.json" , "r", encoding="utf-8") as f:
    lyashevskaya_freq_dict = json.load(f)

In [11]:
def get_top_similar_vectors_from_marked_db(analyzed_text_map,marked_text_db, marked_sent_db, marked_word_db):
    
    similarity_json = {"similar_texts":[],"simiar_sentences":[], "similar_collocations":[]}
    #text_similarity
    text_vector = []
    text_vector.append(analyzed_text_map['lix'])
    text_vector.append(analyzed_text_map['ttr'])
    text_vector.append(analyzed_text_map['sentences_count'])
    text_vector.append(analyzed_text_map['average_sentence_length'])
    text_vector.extend(analyzed_text_map['sent_properties'])
    list_of_texts_similarities = []
    for text_ind in range(len(marked_text_db)):
        #print(text_vector)
        #target_var = list(marked_text_db.iloc[2][-1:]) WILL NEED WHEN PREDICT!
        v1 = np.array(text_vector).reshape(1, -1)
        v2 = np.array(list(marked_text_db.iloc[text_ind][:-1])).reshape(1, -1)
        sim = cosine_similarity(v1,v2)
        #print(sim[0][0])
        if sim[0][0] > 0.6:
            list_of_texts_similarities.append((sim[0][0], {"TEXT_IND_in_marked_db":text_ind, "VECTOR":list(marked_text_db.iloc[text_ind][:-1])}))
            
    list_of_texts_similarities.sort(key = operator.itemgetter(0), reverse= True)   
    similarity_json['similar_texts'] = list_of_texts_similarities[:5]
    
    list_of_all_sentences_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        current_sentence = []
        current_sentence.append(sentence['spec_sentence_features']['negation'])
        current_sentence.append(sentence['spec_sentence_features']['coreference'])
        current_sentence.append(sentence['spec_sentence_features']['vozvr_verb'])
        current_sentence.append(sentence['spec_sentence_features']['prich'])
        current_sentence.append(sentence['spec_sentence_features']['deepr'])
        current_sentence.append(sentence['spec_sentence_features']['case_complexity'])
        current_sentence.append(sentence['spec_sentence_features']['mean_depend_length'])
        v1 = np.array(current_sentence).reshape(1, -1)
        list_of_current_sent_similarities = []
        for sent_ind in range(len(marked_sent_db)):
            v2 = np.array(list(marked_sent_db.iloc[sent_ind][:-1])).reshape(1, -1)
            sim = cosine_similarity(v1,v2)
            if sim[0][0] > 0.6:
                list_of_current_sent_similarities.append((sim[0][0], {"sent_ind_in_marked_db":sent_ind, "sent_vector":list(marked_sent_db.iloc[sent_ind][:-1])}))
        list_of_current_sent_similarities.sort(key = operator.itemgetter(0), reverse= True)
        list_of_all_sentences_similarity.append(list_of_current_sent_similarities[:5])
    similarity_json['simiar_sentences'] = list_of_all_sentences_similarity
    
    list_of_all_words_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        for colloc_word in sentence['collocation_vectors_list']:
            if colloc_word[1][0] in colloc_sim_db:
                similar_words_list = colloc_sim_db[colloc_word[1][0]]
            else:
                similar_words_list = []
            """
            current_word_similarity = []
            #print(colloc[1][0])
            freq_vec = np.array([colloc[1][1], colloc[1][2]]).reshape(1,-1)
            colloc_w2v = np.array(colloc[2]).reshape(1,-1)
            for db_word_index in range(len(marked_word_db)):
                compared_raw_word_vect = marked_word_db.iloc[db_word_index][:-1]
                
                comp_word2vec = compared_raw_word_vect[2:]
                #print(db_word_index, len(comp_word2vec))
                comp_colloc_w2v = np.array(comp_word2vec).reshape(1,-1)
                w2v_sim = cosine_similarity(colloc_w2v,comp_colloc_w2v)
                #print("w2v", sim)
                
               
                comp_freq_vec = np.array([compared_raw_word_vect[0], compared_raw_word_vect[1]]).reshape(1,-1)
                sim_freq = cosine_similarity(freq_vec,comp_freq_vec)
                #print("freq", sim_freq)
                
                w2v_sim = w2v_sim[0][0]
                sim_freq = sim_freq[0][0]
                
                average_similarity = w2v_sim*0.8 + sim_freq*0.2
                
                word_comp_json = {"colloc_db_index":db_word_index, "sim_colloc":words_indexed_dict[db_word_index],"w2v_sim":w2v_sim,"freqence_similarity":sim_freq}
                if average_similarity> 0.6:
                    current_word_similarity.append((word_comp_json,average_similarity))
            current_word_similarity.sort(key = operator.itemgetter(1), reverse= True)
            """
            list_of_all_words_similarity.append((colloc_word[1][0],similar_words_list))
    similarity_json['similar_collocations'] = list_of_all_words_similarity
    return similarity_json

In [14]:
examined_text = get_text_map(texts_lenta.iloc[69]['text'],
                             unigramm_db_path = "./colloc/music_unigr_freq.json",
                            colloc_db_path = "/Users/nigula/Desktop/long_term_storage/colloc/music_smart_colloc_freq.json",
                             raw_text_input=True)
text_sim_list = get_top_similar_vectors_from_marked_db(examined_text,text_db,sent_db,word_db)

In [15]:
text_sim_list['similar_collocations']

[('четыре',
  [[{'marked_colloc_db_index': 197,
     'sim_colloc': 'восемь',
     'w2v_sim': 0.901969158646315,
     'freqence_similarity': 0.9964683770647029},
    0.9208690023299927]]),
 ('рано', []),
 ('не издавать песня', []),
 ('британский поп группа spice', []),
 ('girls', []),
 ('просачиваться',
  [[{'marked_colloc_db_index': 351,
     'sim_colloc': 'разлаживаться',
     'w2v_sim': 0.7022991583255064,
     'freqence_similarity': 0.7619393177594594},
    0.7142271902122971]]),
 ('в интернет', []),
 ('сообщать variety', []),
 ('композиция',
  [[{'marked_colloc_db_index': 192,
     'sim_colloc': 'композиция',
     'w2v_sim': 1.0000000000000009,
     'freqence_similarity': 0.9999999999999998},
    1.0000000000000007]]),
 ('a day in', []),
 ('your', []),
 ('life', []),
 ('pain', []),
 ('proof', []),
 ('if', []),
 ('its', []),
 ('lovin', []),
 ('on', []),
 ('your', []),
 ('mind', []),
 ('и', []),
 ('right', []),
 ('back', []),
 ('at', []),
 ('ya', []),
 ('появляться на', []),
 ('онлай

In [18]:
handel_texts_index = ['0',
 '17',
 '26',
 '32',
 '98',
 '121',
 '130',
 '133',
 '231',
 '200',#!!                     
 '240',
 '316',
 '331',
 '334',
 '336',
 '366',
 '371']
for text_ind in tqdm(range(len(texts_lenta))):
    if str(text_ind) not in handel_texts_index:
        examined_text = get_text_map(texts_lenta.iloc[text_ind]['text'],
                                     unigramm_db_path = "./colloc/music_unigr_freq.json",
                            colloc_db_path = "/Users/nigula/Desktop/long_term_storage/colloc/music_smart_colloc_freq.json", raw_text_input=True)
        text_sim_list = get_top_similar_vectors_from_marked_db(examined_text,text_db,sent_db,word_db)
        with open(os.path.join('./music_similarity',str(text_ind) + ".json"), "w", encoding = 'utf-8') as f:
            json.dump(text_sim_list, f, indent = 2, ensure_ascii=False)
            
            
#"/Users/nigula/Desktop/long_term_storage/colloc/music_smart_colloc_freq.json"
#"D:\input\music_smart_colloc_freq.json"


  2%|▏         | 80/5099 [10:43<12:10:32,  8.73s/it]

[] none in fasttext



  4%|▍         | 207/5099 [30:03<11:39:13,  8.58s/it]

[] none in fasttext



  4%|▍         | 210/5099 [30:47<16:04:30, 11.84s/it]

...


 98%|█████████▊| 5012/5099 [10:57:34<09:29,  6.54s/it]

[] none in fasttext



 98%|█████████▊| 5016/5099 [10:58:05<09:27,  6.84s/it]

[] none in fasttext
[] none in fasttext



100%|██████████| 5099/5099 [11:06:55<00:00,  6.75s/it]

In [ ]:
len(text_sim_list['similar_texts'])

In [ ]:
len(text_sim_list['simiar_sentences'][1])

In [ ]:
text_sim_list['simiar_sentences']

In [ ]:
for sim in text_sim_list['similar_collocations']:
    print (sim)
    print()

# В ПРОЦЕСЕ РЕКОММЕНДАЦИИ ПОЛЬЗОВАТЬСЯ ИНДЕКСАМИ РАЗМЕЧЕННЫХ ПРЕДЛОЖЕНИЙ ИЗ БАЗЫ ДАННЫХ
ТО ЕСТЬ В КАРТАХ ИМИЛАРИТИ ДОЛЖНЫ БЫТЬ СРАЗУ ИНДЕКСЫ ВСЕЙ ХЕРНИ

In [ ]:
for text in text_sim_list:
    if text[1] == text_vector:
        print("yes")